In [1]:
# System information:
# - Linux Mint 18.1 Cinnamon 64-bit
# - Python 2.7 with OpenCV 3.2.0

import numpy as np
import cv2
from cv2 import aruco
import pickle
import glob
import math
import os 

# ChAruco board variables
# CHARUCOBOARD_ROWCOUNT = 12
# CHARUCOBOARD_COLCOUNT = 9 
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_4X4_250)
# ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_6X6_250)


# Create constants to be passed into OpenCV and Aruco methods
# CHARUCO_BOARD = aruco.CharucoBoard_create(
#         squaresX=14,
#         squaresY=10,
#         squareLength=0.01,
#         markerLength=0.007,
#         dictionary=ARUCO_DICT)
# board = aruco.CharucoBoard_create(9, 12, 0.02, 0.015, aruco_dict)
CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=9,
        squaresY=12,
        squareLength=0.02,
        markerLength=0.015,
        dictionary=ARUCO_DICT)




# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = [] # Corners discovered in all images processed
ids_all = [] # Aruco ids corresponding to corners discovered
image_size = None # Determined at runtime
imgs=[]

# This requires a set of images or a video taken with the camera you want to calibrate
# I'm using a set of images taken with the camera with the naming convention:
# 'camera-pic-of-charucoboard-<NUMBER>.jpg'
# All images used should be the same size, which if taken with the same camera shouldn't be a problem
images = glob.glob('picture/8_3_high2/*_color.jpg')
# images = glob.glob('./pic3/*_color.jpg')
print(images)



# Loop through images glob'ed
for iname in images:
    
    base_name = os.path.basename(iname)
    
    # Open the image
    img = cv2.imread(iname)
    # Grayscale the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # print(gray.shape)

    # Find aruco markers in the query image
    corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT)
        # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares
    if len(corners) > 10:
    # Outline the aruco markers found in our query image
        img = aruco.drawDetectedMarkers(
                image=img, 
                corners=corners)

        # Get charuco corners and ids from detected aruco markers
        response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
                markerCorners=corners,
                markerIds=ids,
                image=gray,
                board=CHARUCO_BOARD)


        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)
        
        # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = aruco.drawDetectedCornersCharuco(
                image=img,
                charucoCorners=charuco_corners,
                charucoIds=charuco_ids)
        
        # If our image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]
    
        # Reproportion the image, maxing width or height at 1000
        proportion = max(img.shape) / 1000.0
        img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
        # Pause to display each image, waiting for key press
        try:
            cv2.imshow(iname, img)
            cv2.waitKey(0)
        except:
            # print("saved")
            cv2.imwrite("output/"+base_name, img)
    else:
        cv2.imshow('Charuco board', img)
        cv2.waitKey(0)
        print("Not able to detect a charuco board in image: {}".format(iname))

    # Destroy any open CV windows
    # cv2.destroyAllWindows()

# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure
    exit()

# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)
    
# Print matrix and distortion coefficient to the console
print(cameraMatrix)
print(distCoeffs)
    
# Save values to be used where matrix+dist is required, for instance for posture estimation
# I save files in a pickle file, but you can use yaml or whatever works for you
f = open('calibration.pckl', 'wb')
pickle.dump((cameraMatrix, distCoeffs, rvecs, tvecs), f)
f.close()
    
# Print to console our success
print('Calibration successful. Calibration file used: {}'.format('calibration.pckl'))
print("error:",calibration)

['picture/8_3_high2/020_color.jpg', 'picture/8_3_high2/002_color.jpg', 'picture/8_3_high2/022_color.jpg', 'picture/8_3_high2/004_color.jpg', 'picture/8_3_high2/003_color.jpg', 'picture/8_3_high2/019_color.jpg', 'picture/8_3_high2/008_color.jpg', 'picture/8_3_high2/013_color.jpg', 'picture/8_3_high2/011_color.jpg', 'picture/8_3_high2/006_color.jpg', 'picture/8_3_high2/012_color.jpg', 'picture/8_3_high2/015_color.jpg', 'picture/8_3_high2/016_color.jpg', 'picture/8_3_high2/021_color.jpg', 'picture/8_3_high2/005_color.jpg', 'picture/8_3_high2/018_color.jpg', 'picture/8_3_high2/009_color.jpg', 'picture/8_3_high2/017_color.jpg', 'picture/8_3_high2/010_color.jpg', 'picture/8_3_high2/007_color.jpg', 'picture/8_3_high2/001_color.jpg', 'picture/8_3_high2/014_color.jpg']
[[919.93331992   0.         641.44788625]
 [  0.         924.95271296 352.86463657]
 [  0.           0.           1.        ]]
[[ 0.14960481 -0.24035551  0.00055983 -0.00432475 -0.28952792]]
Calibration successful. Calibration fi

In [8]:
def AngleAxis2EulerZYX(rvec):
    
    R=cv2.Rodrigues(rvec)
    return Mat2Euler(R[0])

def Mat2Euler(_R):
#     sy = math.sqrt(_R.at<double>(0,0) * _R.at<double>(0,0) +  _R.at<double>(1,0) * _R.at<double>(1,0) );
#     sy = math.sqrt(_R[0,0] * _R[0,0] +  _R[0,1] * _R[0,1] );
    sy = math.sqrt(_R[0,0] * _R[0,0] +  _R[1,0] * _R[1,0] );
    if (sy < 1e-6): # If
        singular=True
    else:
        singular=False
#     float x, y, z;
    if (not singular):
#         x = atan2(_R.at<double>(2,1) , _R.at<double>(2,2));
#         y = atan2(-_R.at<double>(2,0), sy);
#         z = atan2(_R.at<double>(1,0), _R.at<double>(0,0));
#         x = math.atan2(_R[1,2] , _R[2,2]);
#         y = math.atan2(-_R[0,2], sy);
#         z = math.atan2(_R[0,1], _R[0,0]);
        x = math.atan2(_R[2,1] , _R[2,2]);
        y = math.atan2(-_R[2,0], sy);
        z = math.atan2(_R[1,0], _R[0,0]);
    else:
#         x = atan2(-_R.at<double>(1,2), _R.at<double>(1,1));
#         y = atan2(-_R.at<double>(2,0), sy);
#         z = 0;
#         x = math.atan2(-_R[2,1], _R[1,1]);
#         y = math.atan2(-_R[0,2], sy);
#         z = 0;
        x = math.atan2(-_R[1,2], _R[1,1]);
        y = math.atan2(-_R[2,0], sy);
        z = 0;
    
    return [x, y, z]


In [10]:
# charucoPose=[]
for i in range(len(images)):
    #ret2,rvec2,tvec2=aruco.estimatePoseCharucoBoard(allCorners,allIds, board,mtx,dist,rvecs,tvecs)
    ret2,rvec2,tvec2=aruco.estimatePoseCharucoBoard(corners_all[i],ids_all[i], CHARUCO_BOARD,cameraMatrix,distCoeffs,rvecs[i],tvecs[i])
    img0 = cv2.imread(images[i])
    imgNew=aruco.drawAxis(img0,cameraMatrix,distCoeffs,rvec2,tvec2,0.1)
    # Reproportion the image, maxing width or height at 1000
    proportion = max(imgNew.shape) / 1000.0
    imgNew = cv2.resize(imgNew, (int(imgNew.shape[1]/proportion), int(imgNew.shape[0]/proportion)))
    # cv2.imshow('Charuco board Axis', imgNew)
    #cv2.waitKey(0)
    Euler = AngleAxis2EulerZYX(rvec2);
    tx = (CHARUCO_BOARD.getChessboardSize()[0]) * (CHARUCO_BOARD.getSquareLength())/2
    ty = (CHARUCO_BOARD.getChessboardSize()[1]) * (CHARUCO_BOARD.getSquareLength())/2
    # TransO = (Mat_<double>(3,1) <<tx,ty,0);
    TransO = np.array([tx,ty,0])

    R=cv2.Rodrigues(rvec2)
    Trans_tvec = R[0].dot(TransO)
    CharucoPose=[0,0,0,0,0,0]
    for j in range(3):
        tvec2[j]+=Trans_tvec[j]
    #     tvec[i]=tvec[i]+Trans_tvec.at<double>(i,0)
        CharucoPose[j]  = tvec2[j][0]*1000
        CharucoPose[j+3]= Euler[j]*180/math.pi
    img01 = cv2.imread(images[i])
    imgNew=aruco.drawAxis(img01,cameraMatrix,distCoeffs,rvec2,tvec2,0.1)
    # Reproportion the image, maxing width or height at 1000
    proportion = max(imgNew.shape) / 1000.0
    imgNew = cv2.resize(imgNew, (int(imgNew.shape[1]/proportion), int(imgNew.shape[0]/proportion)))
    print("imgs:",images[i])
    print("charucoPose:",CharucoPose)
    #saveCamP(CharucoPose,"./pic_highres")
    # cv2.imshow('Charuco board Axis'+str(i), imgNew)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


('imgs:', 'picture/8_3_high2/020_color.jpg')
('charucoPose:', [-7.930926167750499, 49.566171286900214, 423.4892228964707, 174.4401856118526, -11.5510596349129, -90.6337140015373])
('imgs:', 'picture/8_3_high2/002_color.jpg')
('charucoPose:', [-20.120865881287838, 27.675694935843694, 375.23670172711235, -176.1666933797848, -0.6060595100018583, -91.82884532485089])
('imgs:', 'picture/8_3_high2/022_color.jpg')
('charucoPose:', [-43.37007301583695, 40.24688498236356, 409.82530177269007, -165.37915006010638, -9.512896349070163, -94.32040616503937])
('imgs:', 'picture/8_3_high2/004_color.jpg')
('charucoPose:', [-10.707957602144615, 27.62894704137708, 385.7328980328934, -176.12977672927275, -0.5605109405504283, -91.83741773410031])
('imgs:', 'picture/8_3_high2/003_color.jpg')
('charucoPose:', [-10.157486060056744, 27.587372355637825, 375.7598885932697, -176.13622081430552, -0.5926145504041221, -91.84898968137769])
('imgs:', 'picture/8_3_high2/019_color.jpg')
('charucoPose:', [22.1902407912227

In [11]:
ret3,rvec3,tvec3=aruco.estimatePoseCharucoBoard(corners_all[0],ids_all[0], CHARUCO_BOARD,cameraMatrix,distCoeffs,np.array([.0,.0,.0]),np.array([.0,.0,.0]))

# ret3,rvec3,tvec3=aruco.estimatePoseCharucoBoard(corners_all[1],ids_all[1], CHARUCO_BOARD,cameraMatrix,distCoeffs,rvecs[10],tvecs[10])
print("t:",tvec3)
print("r:",rvec3)
Euler = AngleAxis2EulerZYX(rvec3);
# Mat TransO,R;
tx = (CHARUCO_BOARD.getChessboardSize()[0]) * (CHARUCO_BOARD.getSquareLength())/2
ty = (CHARUCO_BOARD.getChessboardSize()[1]) * (CHARUCO_BOARD.getSquareLength())/2
# TransO = (Mat_<double>(3,1) <<tx,ty,0);
TransO = np.array([tx,ty,0])

R=cv2.Rodrigues(rvec3)
print(R[0])
_R=R[0]
print((_R[2,1] , _R[2,2]))
# Trans_tvec = R * TransO    C++
Trans_tvec = R[0].dot(TransO)
print("trans",Trans_tvec)

CharucoPose=[0,0,0,0,0,0]
for i in range(3):
    tvec3[i]+=Trans_tvec[i]
#     tvec[i]=tvec[i]+Trans_tvec.at<double>(i,0)
#     CharucoPose[i]  = tvec3[i][0]*1000
    CharucoPose[i]  = tvec3[i]*1000
    CharucoPose[i+3]= Euler[i]*180/math.pi
imgs=images
img0 = cv2.imread(images[0])
# img0 = cv2.imread(imgs[1])
gray2 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)

# print("new T",tvec3)
print(imgs[0])
print("charucoPose:",CharucoPose)
#tvec3=np.array([-0.0193377,0.0149725,0.374973])
#tvec3=np.array([-0.0193044,0.0249437,0.37477])
imgNew=aruco.drawAxis(img0,cameraMatrix,distCoeffs,rvec3,tvec3,0.1)
# Reproportion the image, maxing width or height at 1000
proportion = max(imgNew.shape) / 1000.0
imgNew = cv2.resize(imgNew, (int(imgNew.shape[1]/proportion), int(imgNew.shape[0]/proportion)))
cv2.imshow('Charuco board Axis', imgNew)
cv2.waitKey(0)
cv2.destroyAllWindows()
#charucoPose: [31.888030030913367, -15.20895736764001, 382.3111896733516, 178.9498020632599, 9.316028012317686, -91.84316482519193]

('t:', array([0.11244674, 0.1340891 , 0.39407681]))
('r:', array([ 2.04899095, -2.09223275,  0.10717839]))
[[-0.01083616 -0.99502015 -0.09908321]
 [-0.97968672  0.03040728 -0.19821536]
 [ 0.20024112  0.09492261 -0.97513752]]
(0.09492260883915539, -0.9751375240182594)
('trans', array([-0.12037767, -0.08452293,  0.02941241]))
picture/8_3_high2/020_color.jpg
('charucoPose:', [-7.930926167750499, 49.566171286900214, 423.4892228964707, 174.4401856118526, -11.5510596349129, -90.6337140015373])


error: /io/opencv/modules/highgui/src/window.cpp:583: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvShowImage


In [13]:
ret2,rvec2,tvec2=aruco.estimatePoseCharucoBoard(corners_all[0],ids_all[0], CHARUCO_BOARD,cameraMatrix,distCoeffs,rvecs[0],tvecs[0])
# aruco::drawAxis(imageCopy, cameraMatrix, distCoeffs, rvec, tvec, 0.1f)
img0 = cv2.imread(images[0])
print("t:",tvec2)
print("r:",rvec2)
imgNew=aruco.drawAxis(img0,cameraMatrix,distCoeffs,rvec2,tvec2,0.1)
# Reproportion the image, maxing width or height at 1000
proportion = max(imgNew.shape) / 1000.0
imgNew = cv2.resize(imgNew, (int(imgNew.shape[1]/proportion), int(imgNew.shape[0]/proportion)))
# cv2.imshow('Charuco board Axis', imgNew)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("output/test.jpg", imgNew)

('t:', array([[0.11244674],
       [0.1340891 ],
       [0.39407681]]))
('r:', array([[ 2.04899095],
       [-2.09223275],
       [ 0.10717839]]))


True